In [2]:
#import all the necessory libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from mpl_toolkits import mplot3d

### K-Means clustering 

In [3]:
#Normalizing function
def normalize(vector):
    for i in range(0,(vector.shape[1]-1)):
        temp=vector.iloc[:,i].to_numpy()
        vector.iloc[:,i]=(temp-np.mean(temp))/np.std(temp)
    return vector

In [4]:
root="/home/subhash/Desktop/spam/nnfl assignment/"
data=pd.read_excel(os.path.join(root,'data4.xlsx'),header=None)
data=normalize(data)
data=data.to_numpy()
data[:,7]=data[:,7]-1
np.random.shuffle(data)
print(data.shape)
#now we obtain the shuffled set of data convinient to split

(150, 8)


In [5]:
#splitting training and testing data with equal number of class data points
def split_train_test(data,k):
    folds=np.array_split(data,k)
    return folds

In [6]:
def split_classes(data):
    y=data[:,-1]
    class0=[]
    class1=[]
    class2=[]
    classes=[]
    for insts in range(0,data.shape[0]):
        if (int(y[insts])==0):
            class0.append(data[insts])
        elif (int(y[insts])==1):
            class1.append(data[insts])
        else :
            class2.append(data[insts])
    c0=np.array(class0)
    c1=np.array(class1)
    c2=np.array(class2)
    classes.append(c0)
    classes.append(c1)
    classes.append(c2)
    return classes

In [7]:
#splitting the data into features and class value
def split_x_y(data):
    y=(data[:,7]).reshape((data.shape[0],1))
    x=np.delete(data,7,1)
    return x,y

In [8]:
#sigmoid function, takes in a single numerical value
def sigmoid(x):
    y=1/(1+np.exp(-x))
    return y
#returns a single numerical value

In [9]:
def hypothesis(w,x):
    hyps=[]
    for insts in range(0,x.shape[0]):
        h=np.dot(w.T,x[insts])
        hyp=sigmoid(h)
        hyps.append(hyp)
    return hyps

In [10]:
#likelyhood function
def likely_func(hyps,y):
    likes=[]
    for insts in range(0,y.shape[0]):
        like=(y[insts]*np.log(hyps[insts]))+((1-y[insts])*np.log(1-hyps[insts]))
        likes.append(like)
    return(np.array(likes))

In [11]:
def costavg(errors):
    costav=np.mean(errors)
    return(costav)

In [12]:
#derivative of likelyhood function
def dervlikef(hyps,y,x):
    deriv=[]
    for insts in range(0,x.shape[0]):
        hyps[insts]=np.reshape((hyps[insts]),(1,1))
        derv=np.dot(np.reshape((x[insts]),(1,7)).T,((y[insts]*(1-hyps[insts])-((1-y[insts])*(hyps[insts])))))
        deriv.append(derv)
        dW=np.squeeze(np.mean(deriv,axis=0,keepdims=True)/2)
        dW=np.reshape(dW,(x.shape[1],1))
    return dW

In [13]:
def update_parameters(w,dervlikef,alpha):
    w=w+(alpha*dervlikef)
    return w

In [14]:
#ForOneVsAll
def create_model(train,class_value):
    model_insts=[]
    count=0
    classes=split_classes(train)
    model_insts.append(classes[class_value])
    for cls in range(0,len(classes)):
        if cls!=class_value:
            np.random.shuffle(classes[cls])
            model_insts.append(classes[cls][:15,:])
    model=np.concatenate((model_insts[0],model_insts[1],model_insts[2]),axis=0)
    for insts in range(0,model.shape[0]):
        if int(model[insts][-1])==class_value:
            model[insts][-1]=1
        else:
            model[insts][-1]=0
    np.random.shuffle(model)
    return model

In [20]:
def train(data,alpha,iters,k):
    folds=split_train_test(data,k)
    overall=[]
    result=[]
    sens=[]
    spec=[]
    acc=[]
    ovaccu=[]
    for fold in range(0,k):
        sensitivity=[]
        specificity=[]
        accuracy=[]
        train=[]
        for fld in range(0,k):
            if (fold==fld):
                test=folds[fold]
            else:
                train.append(folds[fold])
        train=np.array(train).reshape((int(data.shape[0]*(1-k)/k),8))
        test=np.array(test).reshape((int(data.shape[0]/k),8))
        x_train,y_train=split_x_y(train)
        x_test,y_test=split_x_y(test)
        for mod in range(0,3):
            y_temp=[]
            w=np.ones([7,1])
            for i in range(0,iters):
                for insts in range(0,(create_model(train,mod)).shape[0]):
                    mod_train=create_model(train,mod)
                    mod_x_train,mod_y_train=split_x_y(mod_train)
                    deriv=dervlikef(hypothesis(w,mod_x_train),mod_y_train,mod_x_train)
                    w=update_parameters(w,deriv,alpha)
            y_cap=hypothesis(w,x_test)
            result.append(y_cap)
            for insts in range(0,np.array(y_cap).shape[0]):
           
                if (y_cap[insts]<0.5):
                    y_cap[insts]=0
                else:
                    y_cap[insts]=1
            for insts in range(0,y_test.shape[0]):
                if y_test[insts]==mod:
                    y_temp.append(1)
                else:
                    y_temp.append(0)
            y_temp=np.array(y_temp).reshape(y_test.shape)
            y_cap=np.array(y_cap).reshape(y_test.shape)
            true_pos=0
            true_neg=0
            false_pos=0
            false_neg=0
            for insts in range(0,y_test.shape[0]):
                if (y_temp[insts]==1):
                    if (y_cap[insts]==1):
                        true_pos=true_pos+1
                    else:
                        false_neg=false_neg+1
                else:
                    if (y_cap[insts]==0):
                        true_neg=true_neg+1
                    else:
                        false_pos=false_pos+1
            sensitivity.append(true_pos/(true_pos+false_neg))
            specificity.append(true_neg/(true_neg+false_pos))
            accuracy.append((true_pos+true_neg)/(true_pos+true_neg+false_pos+false_neg))
        sens.append(sensitivity)
        spec.append(specificity)
        acc.append(accuracy)
        results=np.array(result)
        results=np.concatenate((results[0].reshape((30,1)),results[1].reshape((30,1)),results[2].reshape((30,1))),1)
        count=0
        for insts in range(y_cap.shape[0]):
            oval=np.argmax(results[insts])
            if oval==y_test[insts]:
                count=count+1
        ovacc=count/(y_cap.shape[0])
        ovaccu.append(ovacc)
    return sens,spec,acc,ovaccu
        

In [21]:
sens,spec,acc,ovacc=train(data,0.05,3,5)

In [22]:
print(sens)
print(spec)
print(acc)
print(ovacc)

[[1.0, 0.8888888888888888, 1.0], [1.0, 1.0, 0.8888888888888888], [1.0, 0.8888888888888888, 0.875], [1.0, 0.8888888888888888, 1.0], [1.0, 1.0, 1.0]]
[[0.95, 0.47619047619047616, 0.8421052631578947], [0.9047619047619048, 0.5, 0.6190476190476191], [1.0, 0.7619047619047619, 0.9090909090909091], [1.0, 0.7619047619047619, 0.7777777777777778], [1.0, 0.47368421052631576, 0.65]]
[[0.9666666666666667, 0.6, 0.9], [0.9333333333333333, 0.7, 0.7], [1.0, 0.8, 0.9], [1.0, 0.8, 0.8666666666666667], [1.0, 0.6666666666666666, 0.7666666666666667]]
[0.5666666666666667, 0.4, 0.43333333333333335, 0.23333333333333334, 0.36666666666666664]
